# Advanced Power Monitoring Examples

This notebook demonstrates advanced usage of the Power-aware HPC Benchmarking project, including:

1. Custom Power Monitor Implementation
2. Integration with HPC Workloads
3. Advanced Data Analysis
4. Power-Performance Optimization
5. Report Generation

## Setup

First, let's import all necessary modules:

In [1]:
import sys
import os
import time
import json
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional

# Add project root to Python path
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

# Import project modules
from src.power_profiling.utils.logging_config import setup_logging
from src.power_profiling.monitors.base import BasePowerMonitor
from src.power_profiling.monitors.cpu import CPUMonitor
from src.power_profiling.monitors.gpu import GPUMonitor
from src.power_profiling.monitors.system import SystemMonitor
from src.power_profiling.utils.power_reading import PowerReading

# Setup logging
logger = setup_logging(log_level=logging.INFO)

ModuleNotFoundError: No module named 'src.power_profiling.utils.power_reading'

## 1. Custom Power Monitor Implementation

Let's create a custom power monitor that combines multiple power sources:

In [ ]:
class CombinedPowerMonitor(BasePowerMonitor):
    """A custom power monitor that combines multiple power sources."""
    
    def __init__(self, monitors: Dict[str, BasePowerMonitor], sampling_interval: float = 1.0):
        """Initialize the combined power monitor.
        
        Args:
            monitors: Dictionary of power monitors
            sampling_interval: Sampling interval in seconds
        """
        super().__init__(sampling_interval)
        self.monitors = monitors
        self.logger = logging.getLogger(__name__)
        
    def start(self) -> None:
        """Start all monitors."""
        self.logger.info("Starting combined power monitor")
        for name, monitor in self.monitors.items():
            try:
                monitor.start()
                self.logger.info(f"Started {name} monitor")
            except Exception as e:
                self.logger.error(f"Failed to start {name} monitor: {e}")
        
    def stop(self) -> Dict[str, List[PowerReading]]:
        """Stop all monitors and return combined data."""
        self.logger.info("Stopping combined power monitor")
        combined_data = {}
        
        for name, monitor in self.monitors.items():
            try:
                data = monitor.stop()
                combined_data[name] = data
                self.logger.info(f"Stopped {name} monitor")
            except Exception as e:
                self.logger.error(f"Failed to stop {name} monitor: {e}")
                
        return combined_data
    
    def get_statistics(self) -> Dict[str, Dict[str, float]]:
        """Get statistics from all monitors."""
        stats = {}
        
        for name, monitor in self.monitors.items():
            try:
                monitor_stats = monitor.get_statistics()
                stats[name] = monitor_stats
            except Exception as e:
                self.logger.error(f"Failed to get statistics for {name} monitor: {e}")
                
        return stats

## 2. Integration with HPC Workloads

Create a benchmark wrapper that integrates power monitoring:

In [ ]:
class PowerAwareBenchmark:
    """A benchmark wrapper with integrated power monitoring."""
    
    def __init__(self, name: str, monitor: BasePowerMonitor):
        self.name = name
        self.monitor = monitor
        self.logger = logging.getLogger(__name__)
        
    def run_benchmark(self, workload_func, *args, **kwargs):
        """Run a benchmark with power monitoring.
        
        Args:
            workload_func: Function to benchmark
            *args, **kwargs: Arguments for the workload function
            
        Returns:
            Dictionary containing benchmark results and power data
        """
        self.logger.info(f"Starting benchmark: {self.name}")
        
        # Start monitoring
        self.monitor.start()
        start_time = time.time()
        
        # Run workload
        try:
            result = workload_func(*args, **kwargs)
            success = True
        except Exception as e:
            self.logger.error(f"Benchmark failed: {e}")
            result = None
            success = False
            
        # Stop monitoring
        end_time = time.time()
        power_data = self.monitor.stop()
        
        # Calculate metrics
        execution_time = end_time - start_time
        power_stats = self.monitor.get_statistics()
        
        return {
            'name': self.name,
            'success': success,
            'result': result,
            'execution_time': execution_time,
            'power_data': power_data,
            'power_stats': power_stats
        }

# Example matrix multiplication benchmark
def matrix_multiply(size: int) -> float:
    """Matrix multiplication benchmark.
    
    Args:
        size: Size of matrices
        
    Returns:
        GFLOPS achieved
    """
    A = np.random.random((size, size))
    B = np.random.random((size, size))
    
    start = time.time()
    C = np.dot(A, B)
    end = time.time()
    
    # Calculate GFLOPS (2 operations per element)
    ops = 2 * size * size * size
    time_seconds = end - start
    gflops = (ops / time_seconds) / 1e9
    
    return gflops

# Create monitors
monitors = {
    'cpu': CPUMonitor(sampling_interval=0.1)
}

try:
    monitors['gpu'] = GPUMonitor(sampling_interval=0.1)
except Exception as e:
    logger.warning(f"GPU monitoring not available: {e}")

# Create combined monitor and benchmark
combined_monitor = CombinedPowerMonitor(monitors)
benchmark = PowerAwareBenchmark("MatrixMultiply", combined_monitor)

# Run benchmark with different sizes
sizes = [1000, 2000, 3000]
results = []

for size in sizes:
    print(f"\nRunning benchmark with size {size}x{size}")
    result = benchmark.run_benchmark(matrix_multiply, size)
    results.append(result)
    
    print(f"GFLOPS: {result['result']:.2f}")
    print(f"Execution time: {result['execution_time']:.2f} seconds")
    for source, stats in result['power_stats'].items():
        print(f"{source.upper()} - Avg: {stats['average']:.2f}W, Peak: {stats['peak']:.2f}W")

## 3. Advanced Data Analysis

Analyze the collected power and performance data:

In [ ]:
class PowerAnalyzer:
    """Advanced power data analysis tools."""
    
    def __init__(self, benchmark_results: List[dict]):
        self.results = benchmark_results
        
    def calculate_efficiency(self) -> pd.DataFrame:
        """Calculate power efficiency (GFLOPS/Watt)."""
        data = []
        
        for result in self.results:
            size = int(np.sqrt(result['result']))
            gflops = result['result']
            
            for source, stats in result['power_stats'].items():
                avg_power = stats['average']
                efficiency = gflops / avg_power
                
                data.append({
                    'size': size,
                    'source': source,
                    'gflops': gflops,
                    'avg_power': avg_power,
                    'efficiency': efficiency
                })
                
        return pd.DataFrame(data)
    
    def plot_efficiency(self):
        """Plot power efficiency analysis."""
        df = self.calculate_efficiency()
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Plot 1: GFLOPS vs Power
        sns.scatterplot(data=df, x='avg_power', y='gflops', 
                       hue='source', size='size', ax=ax1)
        ax1.set_title('Performance vs Power Consumption')
        ax1.set_xlabel('Average Power (W)')
        ax1.set_ylabel('Performance (GFLOPS)')
        
        # Plot 2: Efficiency vs Size
        sns.barplot(data=df, x='size', y='efficiency', 
                   hue='source', ax=ax2)
        ax2.set_title('Power Efficiency by Matrix Size')
        ax2.set_xlabel('Matrix Size')
        ax2.set_ylabel('Efficiency (GFLOPS/W)')
        
        plt.tight_layout()
        plt.show()
        
    def analyze_power_trends(self):
        """Analyze power consumption trends."""
        for result in self.results:
            size = int(np.sqrt(result['result']))
            print(f"\nAnalyzing {size}x{size} matrix:")
            
            for source, readings in result['power_data'].items():
                powers = [r.power_watts for r in readings]
                times = [(r.timestamp - readings[0].timestamp).total_seconds() 
                        for r in readings]
                
                # Calculate trends
                z = np.polyfit(times, powers, 1)
                slope = z[0]
                
                print(f"{source.upper()}:")
                print(f"  Power trend: {slope:.2f} W/s")
                print(f"  Variability: {np.std(powers):.2f} W")

# Create analyzer and generate plots
analyzer = PowerAnalyzer(results)
analyzer.plot_efficiency()
analyzer.analyze_power_trends()

## 4. Power-Performance Optimization

Implement power-aware workload optimization:

In [ ]:
class PowerOptimizer:
    """Power-aware workload optimizer."""
    
    def __init__(self, benchmark: PowerAwareBenchmark):
        self.benchmark = benchmark
        self.logger = logging.getLogger(__name__)
        
    def find_optimal_size(self, 
                         size_range: range,
                         target_metric: str = 'efficiency',
                         power_constraint: Optional[float] = None) -> dict:
        """Find optimal matrix size based on target metric.
        
        Args:
            size_range: Range of matrix sizes to test
            target_metric: 'efficiency' or 'performance'
            power_constraint: Maximum allowed power consumption
            
        Returns:
            Dictionary with optimization results
        """
        results = []
        
        for size in size_range:
            self.logger.info(f"Testing size {size}")
            result = self.benchmark.run_benchmark(matrix_multiply, size)
            
            # Calculate metrics
            gflops = result['result']
            avg_power = np.mean([stats['average'] 
                                for stats in result['power_stats'].values()])
            efficiency = gflops / avg_power
            
            # Check power constraint
            if power_constraint and avg_power > power_constraint:
                self.logger.warning(f"Size {size} exceeds power constraint")
                continue
                
            results.append({
                'size': size,
                'gflops': gflops,
                'avg_power': avg_power,
                'efficiency': efficiency
            })
            
        # Find optimal configuration
        if results:
            optimal = max(results, key=lambda x: x[target_metric])
            self.logger.info(f"Optimal configuration found: {optimal}")
            return optimal
        else:
            self.logger.error("No valid configurations found")
            return None

# Create optimizer and find optimal size
optimizer = PowerOptimizer(benchmark)
optimal = optimizer.find_optimal_size(
    size_range=range(1000, 4001, 1000),
    target_metric='efficiency',
    power_constraint=150  # Example: 150W limit
)

if optimal:
    print("\nOptimal Configuration:")
    print(f"Matrix Size: {optimal['size']}x{optimal['size']}")
    print(f"Performance: {optimal['gflops']:.2f} GFLOPS")
    print(f"Power: {optimal['avg_power']:.2f} W")
    print(f"Efficiency: {optimal['efficiency']:.2f} GFLOPS/W")

## 5. Report Generation

Generate a comprehensive HTML report:

In [ ]:
def generate_report(results: List[dict], filename: str = 'power_report.html'):
    """Generate an HTML report with power analysis results."""
    
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Power Analysis Report</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            table {{ border-collapse: collapse; width: 100%; }}
            th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
            th {{ background-color: #f2f2f2; }}
        </style>
    </head>
    <body>
        <h1>Power Analysis Report</h1>
        <p>Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
        
        <h2>Benchmark Results</h2>
        <table>
            <tr>
                <th>Matrix Size</th>
                <th>Performance (GFLOPS)</th>
                <th>Execution Time (s)</th>
                <th>Average Power (W)</th>
                <th>Peak Power (W)</th>
                <th>Efficiency (GFLOPS/W)</th>
            </tr>
    """
    
    for result in results:
        size = int(np.sqrt(result['result']))
        gflops = result['result']
        exec_time = result['execution_time']
        
        # Average across all power sources
        avg_power = np.mean([stats['average'] 
                            for stats in result['power_stats'].values()])
        peak_power = max([stats['peak'] 
                         for stats in result['power_stats'].values()])
        efficiency = gflops / avg_power
        
        html += f"""
            <tr>
                <td>{size}x{size}</td>
                <td>{gflops:.2f}</td>
                <td>{exec_time:.2f}</td>
                <td>{avg_power:.2f}</td>
                <td>{peak_power:.2f}</td>
                <td>{efficiency:.2f}</td>
            </tr>
        """
    
    html += """
        </table>
        
        <h2>Power Source Details</h2>
    """
    
    for result in results:
        size = int(np.sqrt(result['result']))
        html += f"<h3>{size}x{size} Matrix</h3>"
        
        for source, stats in result['power_stats'].items():
            html += f"""
            <h4>{source.upper()}</h4>
            <ul>
                <li>Average Power: {stats['average']:.2f} W</li>
                <li>Peak Power: {stats['peak']:.2f} W</li>
                <li>Total Energy: {stats['total_energy']:.2f} J</li>
            </ul>
            """
    
    html += """
    </body>
    </html>
    """
    
    with open(filename, 'w') as f:
        f.write(html)
    print(f"Report generated: {filename}")

# Generate report
generate_report(results)

## Conclusion

This notebook demonstrated advanced power monitoring and analysis capabilities:

1. Custom power monitor implementation
2. Integration with HPC workloads
3. Advanced data analysis and visualization
4. Power-aware optimization
5. Comprehensive report generation

These techniques can be applied to optimize the power efficiency of your HPC applications.